In [1]:
rm(list = ls())
#setwd('/Users/zhengwei/Desktop/OneDrive - Texas A&M University-Corpus Christi/Research/Huiyan_Sang/DecisionTree/MyPackage/sim1_lin')
#setwd("/Users/zhengwei/OneDrive - Texas A&M University-Corpus Christi/Research/Huiyan_Sang/DecisionTree/MyPackage/sim1_lin")

library(foreach)
library(doMC)
library(coda)
library(MonBartSFM)
library(frontier)
library(truncnorm)
library(sn)
library(MASS)
library(semsfa)
source("otherfuns.R")
source("Bayes_SFM_NHN_Gibbs.R")

Warning message:
“package ‘foreach’ was built under R version 4.2.3”
Loading required package: iterators

Warning message:
“package ‘iterators’ was built under R version 4.2.3”
Loading required package: parallel

Loading required package: micEcon


If you have questions, suggestions, or comments regarding one of the 'micEcon' packages, please use a forum or 'tracker' at micEcon's R-Forge site:
https://r-forge.r-project.org/projects/micecon/

Loading required package: lmtest

Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric



Please cite the 'frontier' package as:
Tim Coelli and Arne Henningsen (2013). frontier: Stochastic Frontier Analysis. R package version 1.1. http://CRAN.R-Project.org/package=frontier.

If you have questions, suggestions, or comments regarding the 'frontier' package, please use a forum or 'tracker' at frontier's R-Forge site:
https://r-forge.r-project.org/projects/frontie

In [2]:
# Set path to your folder
path <- "simulated_data"

# List all CSV files in that folder
files <- list.files(path, pattern = "\\.csv$", full.names = TRUE)

# Read each CSV into a list of data.frames
simulated_datasets_R <- lapply(files, read.csv)

# Access the first dataset
head(simulated_datasets_R[[1]])

,log_X,log_y
,<dbl>,<dbl>
1,2.768552,0.2527462
2,-2.856968,-0.6890037
3,-1.137870,0.5694323
4,2.452089,2.5354790
5,-1.278954,1.4797850
6,-1.649340,0.2519563


In [3]:
#set cores for parallel computing
RNGkind("L'Ecuyer-CMRG")
cores<-strtoi(Sys.getenv('SLURM_CPUS_PER_TASK', unset=1))
registerDoMC(cores)

part=7

res<-foreach(i=61:70) %dopar% {
  cat("i= ",i,'\n')

finished <- FALSE
while(!finished) {
    tryCatch({

  ##########################################
  #    simulate data for linear case       #
  ##########################################
  ## simulate simple data with one x
  beta0=log(2)
  beta1=0.2
  sigmav=0.9
  sigmau=0.2
  nsamp=200

  x_org=simulated_datasets_R[[i]]$log_X
  y_org=simulated_datasets_R[[i]]$log_y


  ########################################
  # Model 1 fit the Bayes Linear Model
  ########################################

  simdt=data.frame(y1=y_org,lnx1=x_org)

  B_burnin=1000
  B_afterburnin=4000

  # B_burnin=2
  # B_afterburnin=20

  res_Bayelin<-Bayeslinear_NHN_fit(B_burnin,
                                   B_afterburnin,
                                   simdt,
                                   beta0_init = 0,
                                   beta_init=beta0,
                                   sigu_init=sigmau,
                                   sigv_init=sigmav,
                                   df_v = 5,
                                   S_v =(5+2)*sigmav^2,
                                   df_u =5,
                                   S_u = (5+2)*sigmau^2)


  B=B_burnin+B_afterburnin
  res_bayes_lin<-na.omit(res_Bayelin$res[B_burnin+1:B,])

  ##############################################
  # Model 2
  # fit SEM-SFM
  # imposing monotonicity restrictions on inputs
  ##############################################
  #first-step: monotone gam, second-step: fan
  dati=data.frame(y=y_org,x=x_org)
  semfit<-semsfa(y~pbm(x,mono="up"),sem.method = "gam.mono",dati)
  sigma_v_semfit=sqrt(semfit$sigma^2/(1+semfit$lambda^2))
  sigma_u_semfit=semfit$lambda*sigma_v_semfit

  ##############################################
  # Model 3
  # fit Mon-BART-SFM
  # imposing monotonicity restrictions on inputs
  ##############################################
  #find initial value for sigma_v and sigma_u

  fit_sfa <-sfa( y1 ~ lnx1,data = simdt)
  gamma <- unname( coef(fit_sfa)["gamma"] )
  sigmaSq <- unname( coef(fit_sfa)["sigmaSq"] )
  sigmaSqU <- gamma * sigmaSq
  sigmaSqV <- ( 1 - gamma ) * sigmaSq
  betavec<-coef(fit_sfa)[1:3]
  sigma_u<-sqrt(sigmaSqU)
  sigma_v<-sqrt(sigmaSqV)



  ysnfit<-selm(y_org ~ 1, family="SN")
  center_Y <- ysnfit@param$dp[1]

  nskip=B_burnin
  ndpost=B_afterburnin


  fitbartsfm<-monbartsfm(
    x.train=as.matrix(x_org),y.train=y_org, x.test=matrix(0.0,0,0),
    sigest=sigma_v,
    sigdf=3,
    sigquant=.95,
    sigu_pre_est=sigma_u,
    sigquant_u=.95,
    k=2.0,
    power=.8, base=.25, #regular bart is 2,.95
    sigmaf=NA,
    lambda=NA,
    fmean = center_Y,
    #fscale=scale_Y,
    ntree=100,
    ndpost, nskip,
    mgsize=50,
    nkeeptrain=ndpost,nkeeptest=ndpost,
    nkeeptestmean=ndpost,
    nkeeptreedraws=ndpost,
    printevery=100
  )



  fitbartsfm_beta0<-monbartsfm_beta0(
    x.train=as.matrix(x_org),y.train=y_org, x.test=matrix(0.0,0,0),
    sigest=sigma_v,
    sigdf=3,
    sigquant=.95,
    sigu_pre_est=sigma_u,
    sigquant_u=.95,
    k=2.0,
    power=.8, base=.25, #regular bart is 2,.95
    sigmaf=NA,
    lambda=NA,
    fmean = center_Y,
    #fscale=scale_Y,
    ntree=100,
    ndpost, nskip,
    mgsize=50,
    nkeeptrain=ndpost,nkeeptest=ndpost,
    nkeeptestmean=ndpost,
    nkeeptreedraws=ndpost,
    printevery=100
  )

  gtest<-geweke.diag(fitbartsfm$sigma_u)
  pval_BART<-1-pnorm(gtest$z)
  # pval_BART=0.5
  gtest_b0<-geweke.diag(fitbartsfm_beta0$sigma_u)
  pval_BART_b0<-1-pnorm(gtest_b0$z)
  # pval_BART_b0=0.5
  finished <- TRUE #if convergence test is passed, then next iteration

  if(pval_BART>0.1&pval_BART_b0>0.1){
    finished <- TRUE #if convergence test is passed, then next iteration
  }

  #######################################
  # 1. Compare RMSE and BIAS            #
  #######################################

  y_tf = exp(beta0)*exp(x_org)^beta1 #((X**w_true)*b_true).numpy() 
  #############################################################################
  #formulas are from Eq. 10 and 11 from                                       #
  #Ferrara and Vidoli 2017 Semiparametric Stochastic frontier models          #
  #############################################################################
    
  RMSE_bayeslin           = 0 #mean(((mean(res_Bayelin$res$beta0_post)+mean(res_Bayelin$res$beta_current)*x_org-y_org)/y_org)^2) # I did not calculate the Bayes lin
  # e_sem = y_org - (semfit$fitted)
  # E_sem_correction = mean(exp(e_sem))

  E_sem_correction = exp((sigma_v_semfit**2+sigma_u_semfit**2)/2)    
  RMSE_sem         = mean(((exp(semfit$fitted)*E_sem_correction-y_tf)/y_tf)^2)
  # e_bartsfm = y_org - (fitbartsfm$yhat.train.mean)
  # E_bartsfm_correction = mean(exp(e_bartsfm))
  sigv_map = median(fitbartsfm$sigma[(nskip+1):(nskip+ndpost)])
  sigu_map = median(fitbartsfm$sigma_u[(nskip+1):(nskip+ndpost)])
  
  E_bartsfm_correction =  exp((sigv_map**2+sigv_map**2)/2)    
  RMSE_bartsfm            =mean(((exp(fitbartsfm$yhat.train.mean)*E_bartsfm_correction-y_tf)/y_tf)^2)
        
  # RMSE_bartsfm_uadj       =mean(((fitbartsfm_uadj$yhat.train.mean-y_org)/y_org)^2)
  # e_bartsfm_beta0 = y_org - (fitbartsfm_beta0$yhat.train.mean)
  # E_bartsfm_beta0_correction = mean(exp(e_bartsfm_beta0))
  sigv_beta0_map = median(fitbartsfm_beta0$sigma[(nskip+1):(nskip+ndpost)])
  sigu_beta0_map = median(fitbartsfm_beta0$sigma_u[(nskip+1):(nskip+ndpost)])

  E_bartsfm_beta0_correction = exp((sigv_beta0_map**2+sigu_beta0_map**2)/2)   
  RMSE_bartsfm_beta0      =mean(((exp(fitbartsfm_beta0$yhat.train.mean)*E_bartsfm_beta0_correction-y_tf)/y_tf)^2)
  # RMSE_bartsfm_beta0_uadj =mean(((fitbartsfm_beta0_uadj$yhat.train.mean-y_org)/y_org)^2)

  BIAS_bayeslin           =mean(abs((mean(res_Bayelin$res$beta0_post)+mean(res_Bayelin$res$beta_current)*x_org-y_org)/y_org))
  BIAS_sem                =mean(abs((exp(semfit$fitted)*E_sem_correction-y_tf)/y_tf))
  BIAS_bartsfm            =mean(abs((exp(fitbartsfm$yhat.train.mean)*E_bartsfm_correction-y_tf)/y_tf))
  # BIAS_bartsfm_uadj       =median(((fitbartsfm_uadj$yhat.train.mean-y_org)/y_org))
  BIAS_bartsfm_beta0      =median(abs((exp(fitbartsfm_beta0$yhat.train.mean)*E_bartsfm_beta0_correction-y_tf)/y_tf))
  # BIAS_bartsfm_beta0_uadj =median(((fitbartsfm_beta0_uadj$yhat.train.mean-y_org)/y_org))

  RMSE_res<-c(RMSE_bayeslin,
              RMSE_sem,
              RMSE_bartsfm,
            # RMSE_bartsfm_uadj,
              RMSE_bartsfm_beta0
  )

  BIAS_res <- c(BIAS_bayeslin,
                BIAS_sem,
                BIAS_bartsfm,
                # BIAS_bartsfm_uadj,
                BIAS_bartsfm_beta0
                # BIAS_bartsfm_beta0_uadj
                )

  ##############################################################################
  # 2. Compare mean bias estimates of sigma_u and sigma_v                           #
  ##############################################################################
  Bias_sig_v_bayeslin           = abs(median(res_bayes_lin$sigv_post)-sigmav)
  Bias_sig_v_sem                = abs(sigma_v_semfit-sigmav)
  Bias_sig_v_bartsfm            = abs(median(fitbartsfm$sigma[(nskip+1):(nskip+ndpost)])      -sigmav)
  # Bias_sig_v_bartsfm_uadj       = abs(median(fitbartsfm_uadj$sigma[(nskip+1):(nskip+ndpost)]) -sigmav)
  Bias_sig_v_bartsfm_beta0      = abs(median(fitbartsfm_beta0$sigma[(nskip+1):(nskip+ndpost)])-sigmav)
  # Bias_sig_v_bartsfm_beta0_uadj = abs(median(fitbartsfm_uadj$sigma[(nskip+1):(nskip+ndpost)]) -sigmav)

  Bias_sig_u_bayeslin           = abs(median(res_bayes_lin$sigu_post)-sigmau)
  Bias_sig_u_sem                = abs(sigma_u_semfit-sigmau)
  Bias_sig_u_bartsfm            = abs(median(fitbartsfm$sigma_u[(nskip+1):(nskip+ndpost)])     -sigmau)
  # Bias_sig_u_bartsfm_uadj       = abs(median(fitbartsfm_uadj$sigma_u[(nskip+1):(nskip+ndpost)])  -sigmau)
  Bias_sig_u_bartsfm_beta0      = abs(median(fitbartsfm_beta0$sigma_u[(nskip+1):(nskip+ndpost)])  -sigmau)
  # Bias_sig_u_bartsfm_beta0_uadj = abs(median(fitbartsfm_uadj$sigma_u[(nskip+1):(nskip+ndpost)])   -sigmau)

  Bias_sig_v_res <- c(
      Bias_sig_v_bayeslin      ,
      Bias_sig_v_sem           ,
      Bias_sig_v_bartsfm       ,
      # Bias_sig_v_bartsfm_uadj  ,
      Bias_sig_v_bartsfm_beta0
      # Bias_sig_v_bartsfm_beta0_uadj
    )

  Bias_sig_u_res <-
    c(Bias_sig_u_bayeslin      ,
      Bias_sig_u_sem           ,
      Bias_sig_u_bartsfm       ,
      # Bias_sig_u_bartsfm_uadj  ,
      Bias_sig_u_bartsfm_beta0
      # Bias_sig_u_bartsfm_beta0_uadj
    )


  ##############################################################################
  # 3. Compare mean inefficiency scores                                        #
  ##############################################################################
  #True TEs
  res_true<-simdt$y1-(beta0+beta1*simdt$lnx1)
  TEvec_true<-sapply(res_true,TE_fun,sigv=sigmav,sigu=sigmau)
  mean(TEvec_true)

  #Bayes regression TEs
  TEvec_bayeslin<-apply(res_bayes_lin,1,function(x){ return(TE_bayes_fun(bayelinres = x,y = simdt$y1,x=simdt$lnx1))})
  mean(TEvec_bayeslin)

  #TE for sem
  res_sem<-simdt$y1-semfit$fitted
  TEvec_sem<-sapply(res_sem,TE_fun,sigv=sigma_v_semfit,sigu=sigma_u_semfit)
  mean(TEvec_sem)

  #TE for bart-sfm
  TEvec_bartsfm<-apply(cbind(fitbartsfm$yhat.train,
                             fitbartsfm$sigma[(nskip+1):(nskip+ndpost)],
                             fitbartsfm$sigma_u[(nskip+1):(nskip+ndpost)]),1,
                       function(y){return(TE_bart_fun(yhat_post = y[1:nsamp],
                                                      sigv_post = y[nsamp+1],
                                                      sigu_post = y[nsamp+2],
                                                      y_bart = simdt$y1
                       ))})

  #TE for bart-sfm-uadj
  # TEvec_bartsfm_uadj<-apply(cbind(fitbartsfm_uadj$yhat.train,
  #                                 fitbartsfm_uadj$sigma[(nskip+1):(nskip+ndpost)],
  #                                 fitbartsfm_uadj$sigma_u[(nskip+1):(nskip+ndpost)]),1,
  #                           function(y){return(TE_bart_fun(yhat_post = y[1:nsamp],
  #                                                          sigv_post = y[nsamp+1],
  #                                                          sigu_post = y[nsamp+2],
  #                                                          y_bart = simdt$y1
  #                           ))})

  #TE for bart-sfm-beta0
  TEvec_bartsfm_beta0<-apply(cbind(fitbartsfm_beta0$yhat.train,
                                   fitbartsfm_beta0$sigma[(nskip+1):(nskip+ndpost)],
                                   fitbartsfm_beta0$sigma_u[(nskip+1):(nskip+ndpost)]),1,
                             function(y){return(TE_bart_fun(yhat_post = y[1:nsamp],
                                                            sigv_post = y[nsamp+1],
                                                            sigu_post = y[nsamp+2],
                                                            y_bart = simdt$y1
                             ))})


  #TE for bart-sfm-beta0-uadj
  # TEvec_bartsfm_beta0_uadj<-apply(cbind(fitbartsfm_beta0_uadj$yhat.train,
  #                                       fitbartsfm_beta0_uadj$sigma[(nskip+1):(nskip+ndpost)],
  #                                       fitbartsfm_beta0_uadj$sigma_u[(nskip+1):(nskip+ndpost)]),1,
  #                                 function(y){return(TE_bart_fun(yhat_post = y[1:nsamp],
  #                                                                sigv_post = y[nsamp+1],
  #                                                                sigu_post = y[nsamp+2],
  #                                                                y_bart = simdt$y1
  #                                 ))})

  TE_bias<-abs(c(
    mean(TEvec_bayeslin),
    mean(TEvec_sem),
    mean(TEvec_bartsfm),
    # mean(TEvec_bartsfm_uadj),
    mean(TEvec_bartsfm_beta0)
    # mean(TEvec_bartsfm_beta0_uadj)
  )-mean(TEvec_true))

    },
  error = function(e) {})
}

  # TE_bias<-rbind(TE_bias,TE_bias_one)

  return(list(RMSE=RMSE_res,
              BIAS=BIAS_res,
              Bias_sig_v=Bias_sig_v_res,
              Bias_sig_u=Bias_sig_u_res,
              TE_bias=TE_bias,
              simdata=simdt,
              sig_v_bart_post=fitbartsfm$sigma,
              sig_u_bart_post=fitbartsfm$sigma_u,
              sig_v_bart_b0_post=fitbartsfm_beta0$sigma,
              sig_u_bart_b0_post=fitbartsfm_beta0$sigma_u
              ))

}

RMSE_res<-as.data.frame(do.call(rbind,lapply(res,function(x){x[[1]]})))
colnames(RMSE_res)<-c("RMSE_BayesLin","RMSE_Sem",
                      "RMSE_Bart",
                      # "RMSE_Bart_uadj",
                      "RMSE_Bart_b0"
                      # "RMSE_Bart_uadj_b0"
)

BIAS_res<-as.data.frame(do.call(rbind,lapply(res,function(x){x[[2]]})))
colnames(BIAS_res)<-c("BIAS_BayesLin","BIAS_Sem",
                      "BIAS_Bart",
                      # "BIAS_Bart_uadj",
                      "BIAS_Bart_b0"
                      # "BIAS_Bart_uadj_b0"
)

Bias_sig_v_res<-as.data.frame(do.call(rbind,lapply(res,function(x){x[[3]]})))
colnames(Bias_sig_v_res)<-c("Sigvb_BayesLin","Sigvb_Sem",
                            "Sigvb_Bart",
                            # "Sigvb_Bart_uadj",
                            "Sigvb_Bart_b0"
                            # "Sigvb_Bart_uadj_b0"
)

Bias_sig_u_res<-as.data.frame(do.call(rbind,lapply(res,function(x){x[[4]]})))
colnames(Bias_sig_u_res)<-c("Sigub_BayesLin",
                            "Sigub_Sem",
                            "Sigub_Bart",
                            # "Sigub_Bart_uadj",
                            "Sigub_Bart_b0"
                            # "Sigub_Bart_uadj_b0"
)

TE_bias<-as.data.frame(do.call(rbind,lapply(res,function(x){x[[5]]})))
colnames(TE_bias)<-c("TEb_BayesLin","TEb_Sem",
                     "TEb_Bart",
                     # "TEb_Bart_uadj",
                     "TEb_Bart_b0"
                     # "TEb_Bart_uadj_b0"
)

resfin<-list(RMSE=RMSE_res,
             BIAS=BIAS_res,
             Bias_sigv=Bias_sig_v_res,
             Bias_sigu=Bias_sig_u_res,
             TE_bias=TE_bias,
             res=res)
saveRDS(resfin, paste("res/linear_case_part",part,".rds",sep = ""))

i=  61 
Cumulative average: iter= 1000 beta0= 0.7195 beta= 0.1935 sigmav= 0.9003 sigmau= 0.2761 
Cumulative average: iter= 2000 beta0= 0.7207 beta= 0.1933 sigmav= 0.8996 sigmau= 0.2757 
Cumulative average: iter= 3000 beta0= 0.7171 beta= 0.1936 sigmav= 0.9004 sigmau= 0.2717 
Cumulative average: iter= 4000 beta0= 0.7179 beta= 0.1931 sigmav= 0.9009 sigmau= 0.2732 
Cumulative average: iter= 5000 beta0= 0.717 beta= 0.1929 sigmav= 0.9004 sigmau= 0.2717 


Warning message in semsfa(y ~ pbm(x, mono = "up"), sem.method = "gam.mono", dati):
“the residuals of the FirstStep estimates are left skewed 
 this might indicate that there is no inefficiency or that the model is misspecified”
Warning message in sfa(y1 ~ lnx1, data = simdt):
“the residuals of the OLS estimates are right-skewed; this might indicate that there is no inefficiency or that the model is misspecified”
Warning message in sfa(y1 ~ lnx1, data = simdt):
“the parameter 'gamma' is close to the boundary of the parameter space [0,1]: this can cause convergence problems and can negatively affect the validity and reliability of statistical tests and might be caused by model misspecification”
Loading required package: Rcpp



This is my mon-Bart code
*****Into main of monotonic bart
**********************
n: 200
p: 1
first and last y: 1.587381, 2.658280
first row: -1.593915, -1.593915
second row: -0.604299, -0.604299
last row: 1.669096, 1.669096
no test observations
tau: 0.145730
nu: 3.000000
lambda: 0.095763
tree prior base: 0.250000
tree prior power: 0.800000
burn (nskip): 1000
nd (ndpost): 4000
m (ntree): 100
nm (mu grid size): 50
*****nkeeptrain,nkeeptest,nkeeptestme, nkeeptreedraws: 4000, 4000, 4000, 4000
*****printevery: 100
*****skiptr,skipte,skipteme,skiptreedraws: 1,1,1,1
**********************
dip.n: 0
x1 cuts: -2.928335 ... 2.931684
check counts
trcnt,tecnt,temecnt,treedrawscnt: 4000,0,0, 4000
This is my mon-Bart code
*****Into main of monotonic bart
**********************
n: 200
p: 1
first and last y: 1.587381, 2.658280
first row: -1.593915, -1.593915
second row: -0.604299, -0.604299
last row: 1.669096, 1.669096
no test observations
tau: 0.145730
nu: 3.000000
lambda: 0.095763
tree prior base: 0.

Warning message in semsfa(y ~ pbm(x, mono = "up"), sem.method = "gam.mono", dati):
“the residuals of the FirstStep estimates are left skewed 
 this might indicate that there is no inefficiency or that the model is misspecified”
Warning message in sfa(y1 ~ lnx1, data = simdt):
“the residuals of the OLS estimates are right-skewed; this might indicate that there is no inefficiency or that the model is misspecified”
Warning message in sfa(y1 ~ lnx1, data = simdt):
“the parameter 'gamma' is close to the boundary of the parameter space [0,1]: this can cause convergence problems and can negatively affect the validity and reliability of statistical tests and might be caused by model misspecification”


This is my mon-Bart code
*****Into main of monotonic bart
**********************
n: 200
p: 1
first and last y: -0.473001, -0.824167
first row: -2.851303, -2.851303
second row: -0.089861, -0.089861
last row: 1.158245, 1.158245
no test observations
tau: 0.149102
nu: 3.000000
lambda: 0.090272
tree prior base: 0.250000
tree prior power: 0.800000
burn (nskip): 1000
nd (ndpost): 4000
m (ntree): 100
nm (mu grid size): 50
*****nkeeptrain,nkeeptest,nkeeptestme, nkeeptreedraws: 4000, 4000, 4000, 4000
*****printevery: 100
*****skiptr,skipte,skipteme,skiptreedraws: 1,1,1,1
**********************
dip.n: 0
x1 cuts: -2.918532 ... 2.936002
check counts
trcnt,tecnt,temecnt,treedrawscnt: 4000,0,0, 4000
This is my mon-Bart code
*****Into main of monotonic bart
**********************
n: 200
p: 1
first and last y: -0.473001, -0.824167
first row: -2.851303, -2.851303
second row: -0.089861, -0.089861
last row: 1.158245, 1.158245
no test observations
tau: 0.149102
nu: 3.000000
lambda: 0.090272
tree prior base

...

This is my mon-Bart code
*****Into main of monotonic bart
**********************
n: 200
p: 1
first and last y: 0.118173, -0.897149
first row: -0.388402, -0.388402
second row: 2.990651, 2.990651
last row: -0.574085, -0.574085
no test observations
tau: 0.197736
nu: 3.000000
lambda: 0.107431
tree prior base: 0.250000
tree prior power: 0.800000
burn (nskip): 1000
nd (ndpost): 4000
m (ntree): 100
nm (mu grid size): 50
*****nkeeptrain,nkeeptest,nkeeptestme, nkeeptreedraws: 4000, 4000, 4000, 4000
*****printevery: 100
*****skiptr,skipte,skipteme,skiptreedraws: 1,1,1,1
**********************
dip.n: 0
x1 cuts: -2.872984 ... 2.932015
check counts
trcnt,tecnt,temecnt,treedrawscnt: 4000,0,0, 4000
This is my mon-Bart code
*****Into main of monotonic bart
**********************
n: 200
p: 1
first and last y: 0.118173, -0.897149
first row: -0.388402, -0.388402
second row: 2.990651, 2.990651
last row: -0.574085, -0.574085
no test observations
tau: 0.197736
nu: 3.000000
lambda: 0.107431
tree prior base: 

Warning message in semsfa(y ~ pbm(x, mono = "up"), sem.method = "gam.mono", dati):
“the residuals of the FirstStep estimates are left skewed 
 this might indicate that there is no inefficiency or that the model is misspecified”
Warning message in sfa(y1 ~ lnx1, data = simdt):
“the residuals of the OLS estimates are right-skewed; this might indicate that there is no inefficiency or that the model is misspecified”
Warning message in sfa(y1 ~ lnx1, data = simdt):
“the parameter 'gamma' is close to the boundary of the parameter space [0,1]: this can cause convergence problems and can negatively affect the validity and reliability of statistical tests and might be caused by model misspecification”


This is my mon-Bart code
*****Into main of monotonic bart
**********************
n: 200
p: 1
first and last y: -0.633182, 1.663987
first row: -2.871334, -2.871334
second row: 1.614425, 1.614425
last row: 2.225869, 2.225869
no test observations
tau: 0.167663
nu: 3.000000
lambda: 0.102575
tree prior base: 0.250000
tree prior power: 0.800000
burn (nskip): 1000
nd (ndpost): 4000
m (ntree): 100
nm (mu grid size): 50
*****nkeeptrain,nkeeptest,nkeeptestme, nkeeptreedraws: 4000, 4000, 4000, 4000
*****printevery: 100
*****skiptr,skipte,skipteme,skiptreedraws: 1,1,1,1
**********************
dip.n: 0
x1 cuts: -2.938630 ... 2.872988
check counts
trcnt,tecnt,temecnt,treedrawscnt: 4000,0,0, 4000
This is my mon-Bart code
*****Into main of monotonic bart
**********************
n: 200
p: 1
first and last y: -0.633182, 1.663987
first row: -2.871334, -2.871334
second row: 1.614425, 1.614425
last row: 2.225869, 2.225869
no test observations
tau: 0.167663
nu: 3.000000
lambda: 0.102575
tree prior base: 0.25

Warning message in semsfa(y ~ pbm(x, mono = "up"), sem.method = "gam.mono", dati):
“the residuals of the FirstStep estimates are left skewed 
 this might indicate that there is no inefficiency or that the model is misspecified”
Warning message in sfa(y1 ~ lnx1, data = simdt):
“the residuals of the OLS estimates are right-skewed; this might indicate that there is no inefficiency or that the model is misspecified”
Warning message in sfa(y1 ~ lnx1, data = simdt):
“the parameter 'gamma' is close to the boundary of the parameter space [0,1]: this can cause convergence problems and can negatively affect the validity and reliability of statistical tests and might be caused by model misspecification”


This is my mon-Bart code
*****Into main of monotonic bart
**********************
n: 200
p: 1
first and last y: 0.209628, 1.281991
first row: 2.488128, 2.488128
second row: -0.714764, -0.714764
last row: 1.833738, 1.833738
no test observations
tau: 0.128531
nu: 3.000000
lambda: 0.083705
tree prior base: 0.250000
tree prior power: 0.800000
burn (nskip): 1000
nd (ndpost): 4000
m (ntree): 100
nm (mu grid size): 50
*****nkeeptrain,nkeeptest,nkeeptestme, nkeeptreedraws: 4000, 4000, 4000, 4000
*****printevery: 100
*****skiptr,skipte,skipteme,skiptreedraws: 1,1,1,1
**********************
dip.n: 0
x1 cuts: -2.933527 ... 2.939687
check counts
trcnt,tecnt,temecnt,treedrawscnt: 4000,0,0, 4000
This is my mon-Bart code
*****Into main of monotonic bart
**********************
n: 200
p: 1
first and last y: 0.209628, 1.281991
first row: 2.488128, 2.488128
second row: -0.714764, -0.714764
last row: 1.833738, 1.833738
no test observations
tau: 0.128531
nu: 3.000000
lambda: 0.083705
tree prior base: 0.2500